In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert about a set of software for building LLM-powered applications called LangChain, LangGraph, LangServe, and LangSmith.

LangChain is a Python framework that provides a large set of integrations that can easily be composed to build LLM applications.
LangGraph is a Python package built on top of LangChain that makes it easy to build stateful, multi-actor LLM applications.
LangServe is a Python package built on top of LangChain that makes it easy to deploy a LangChain application as a REST API.
LangSmith is a platform that makes it easy to trace and test LLM applications.

Answer the user question as best you can. Answer as though you were writing a tutorial that addressed the user question."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
qa_no_context = prompt | llm | StrOutputParser()

In [3]:
answer = qa_no_context.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)
print(answer)

To use multi-modal models in a chain and turn the chain into a REST API, you can leverage the capabilities of LangChain, LangGraph, and LangServe. Here's a step-by-step guide on how to achieve this:

1. **Set up your environment**: Make sure you have Python installed on your system. You can install LangChain, LangGraph, and LangServe using pip:

```bash
pip install langchain langgraph langserve
```

2. **Create a multi-modal model**: You can create a multi-modal model using LangChain by composing different integrations. For example, you can combine text and image processing integrations to build a multi-modal model.

3. **Build a stateful, multi-actor application**: Use LangGraph to build a stateful, multi-actor application that incorporates your multi-modal model. LangGraph simplifies the process of creating complex, stateful applications by providing abstractions for actors, states, and transitions.

4. **Deploy the application as a REST API**: Once you have built your multi-actor ap

In [4]:
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings

In [5]:
base_embeddings = OpenAIEmbeddings()
llm = OpenAI()

In [7]:
multi_llm = OpenAI(n=4, best_of=4)
embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, base_embeddings, "web_search"
)
result = embeddings.embed_query("Where is the Taj Mahal?")
print(result[:10])
print(len(result))

[-0.008367725713174623, -0.005661255873196436, -0.00837634540923301, -0.00023159501630035692, -0.003151094342014292, 0.020679192620382876, -0.034114847140683406, -0.009861742027087811, -0.012565696258124927, -0.012407280390692393]
1536


In [8]:
prompt_template = """Please answer the user's question about the most recent state of the union address
Question: {question}
Answer:"""
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

embedding_fn = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain, base_embeddings=base_embeddings
)

result = embedding_fn.embed_query(
    "What did the president say about Ketanji Brown Jackson"
)

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

with open("data/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [11]:
docsearch = Chroma.from_texts(texts, embedding_fn, persist_directory="./data/.chroma_db")


query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [12]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
